# Happiness Report & CIA Factbook

Data from the Happiness Report combined with data from the CIA World Factbook to find possible correlations

## A big limitation here is that the Happiness Report is actually using data which occurs in the CIA World Factbook (such as GDP per capita or life expectancy), so correlations here are obvious. 

An analysis therefore is actually redundant. This is more fun but it will not provide any new insights.

In [8]:
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

In [9]:
happiness = pd.read_csv('data/2017.csv', dtype={'Country': np.object}).drop(['Happiness.Score', 'Whisker.high', 'Whisker.low', 'Economy..GDP.per.Capita.'], axis=1)
happiness.head()

,Country,Happiness.Rank,Family,Health..Life.Expectancy.,Freedom,Generosity,Trust..Government.Corruption.,Dystopia.Residual
0,Norway,1,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027
1,Denmark,2,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707
2,Iceland,3,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715
3,Switzerland,4,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716
4,Finland,5,1.540247,0.809158,0.617951,0.245483,0.382612,2.430182


In [10]:
factbook = pd.read_csv('data/countries-of-the-world.csv', decimal=',', sep=',', dtype={'Country': np.object})
factbook['Country'] = factbook['Country'].str.strip() #removing the whitespaces after each country in factbook, they are all like 'Afghanistan ', 'Albania ' etc
factbook['Region'] = factbook['Region'].str.strip()
factbook.head()

,Country,Region,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service
0,Afghanistan,ASIA (EX. NEAR EAST),31056997,647500,48.0,0.00,23.06,163.07,700.0,36.0,3.2,12.13,0.22,87.65,1.0,46.60,20.34,0.380,0.240,0.380
1,Albania,EASTERN EUROPE,3581655,28748,124.6,1.26,-4.93,21.52,4500.0,86.5,71.2,21.09,4.42,74.49,3.0,15.11,5.22,0.232,0.188,0.579
2,Algeria,NORTHERN AFRICA,32930091,2381740,13.8,0.04,-0.39,31.00,6000.0,70.0,78.1,3.22,0.25,96.53,1.0,17.14,4.61,0.101,0.600,0.298
3,American Samoa,OCEANIA,57794,199,290.4,58.29,-20.71,9.27,8000.0,97.0,259.5,10.00,15.00,75.00,2.0,22.46,3.27,NaN,NaN,NaN
4,Andorra,WESTERN EUROPE,71201,468,152.1,0.00,6.60,4.05,19000.0,100.0,497.2,2.22,0.00,97.78,3.0,8.71,6.25,NaN,NaN,NaN


## In order to combine both dataframes, I convert both dataframes with the pivot_table function first and concatenate then

In [11]:
factbook_pivot = factbook.pivot_table(columns=['Country'])
happiness_pivot = happiness.pivot_table(columns=['Country'])

In [12]:
data = pd.concat([factbook_pivot, happiness_pivot])#concatenating both pivot tables together
data = data.astype(object)
data = data.T
data = data.dropna(subset=['Happiness.Rank']) #delete all the rows/countries that don't appear in the happiness report
data_plot = data #for plotting later
data = data.dropna() #delete any rows that have no data
data.head()

,Agriculture,Arable (%),Area (sq. mi.),Birthrate,Climate,Coastline (coast/area ratio),Crops (%),Deathrate,GDP ($ per capita),Industry,...,Pop. Density (per sq. mi.),Population,Service,Dystopia.Residual,Family,Freedom,Generosity,Happiness.Rank,Health..Life.Expectancy.,Trust..Government.Corruption.
Afghanistan,0.38,12.13,647500,46.6,1,0,0.22,20.34,700,0.24,...,48,3.1057e+07,0.38,2.1508,0.581543,0.10618,0.311871,141,0.180747,0.0611578
Albania,0.232,21.09,28748,15.11,3,1.26,4.42,5.22,4500,0.188,...,124.6,3.58166e+06,0.579,1.49044,0.803685,0.381499,0.201313,109,0.73116,0.0398642
Algeria,0.101,3.22,2.38174e+06,17.14,1,0.04,0.25,4.61,6000,0.6,...,13.8,3.29301e+07,0.298,2.5676,1.14622,0.233336,0.0694366,53,0.617585,0.146096
Argentina,0.095,12.31,2.76689e+06,16.73,3,0.18,0.48,7.55,11200,0.358,...,14.4,3.99218e+07,0.547,2.61401,1.44045,0.494519,0.109457,24,0.695137,0.0597399
Armenia,0.239,17.55,29800,12.07,4,0,2.3,8.23,3500,0.343,...,99.9,2.97637e+06,0.418,1.5215,1.00748,0.198303,0.0834881,121,0.637524,0.0266744


After preparing the data, there are 121 countries left which are in the Happiness Report AND have no data in the factbook missing

## Let's look for correlations using the pandas .corr function

In [13]:
Happiness_Correlations = data.astype(float).corr(method='pearson', min_periods=10)
Happiness_Correlations = Happiness_Correlations[['Happiness.Rank']]
Happiness_Correlations.abs().sort_values(by='Happiness.Rank', ascending=False)

,Happiness.Rank
Happiness.Rank,1.000000
Health..Life.Expectancy.,0.782471
GDP ($ per capita),0.752629
Family,0.736655
Phones (per 1000),0.731553
Infant mortality (per 1000 births),0.707118
Birthrate,0.679356
Agriculture,0.676177
Literacy (%),0.634532
Freedom,0.588837


We can see that the highest correlation is with GDP per capita (from the Happiness Report) followed by life expectancy (From the Happiness Report) and GDP per Capita again (from the Factbook)

Or in other words, the more money per capita, the more happy a country is. 

Let's show that graphically

## World Graphs

In [14]:
plotly.offline.iplot({
    "data": [go.Scatter(x=data['Happiness.Rank'], 
                        y=data['Health..Life.Expectancy.'], 
                        mode="markers", 
                        hovertext=data.index,
                        )
            ],
    "layout": go.Layout(title="World",
                        xaxis=dict(title='Happiness Rank'),
                        yaxis=dict(title='Life Expectancy')
                       )
})

In [15]:
plotly.offline.iplot({
    "data": [go.Scatter(x=data['Happiness.Rank'], 
                        y=data['GDP ($ per capita)'], 
                        mode="markers", 
                        hovertext=data.index,
                        )
            ],
    "layout": go.Layout(title="World",
                        xaxis=dict(title='Happiness Rank'),
                        yaxis=dict(title='GDP per capita ($)')
                       )
})

In [16]:
plotly.offline.iplot({
    "data": [go.Scatter(x=data['Happiness.Rank'], 
                        y=data['Family'], 
                        mode="markers", 
                        hovertext=data.index,
                        )
            ],
    "layout": go.Layout(title="World",
                        xaxis=dict(title='Happiness Rank'),
                        yaxis=dict(title='Family Factor')
                       )
})

# Regions

Since there are high similarities among the cultures in certain regions, I want to extract regions and look at them seperately. In the newest Happiness Report, there is f.i. a section detailed about South America.

I want to look especially at Europe and South America. 

## Europe

In [17]:
factbook.Region.unique()

array(['ASIA (EX. NEAR EAST)', 'EASTERN EUROPE', 'NORTHERN AFRICA',
       'OCEANIA', 'WESTERN EUROPE', 'SUB-SAHARAN AFRICA',
       'LATIN AMER. & CARIB', 'C.W. OF IND. STATES', 'NEAR EAST',
       'NORTHERN AMERICA', 'BALTICS'], dtype=object)

In [18]:
factbook_europe = factbook.loc[factbook['Region'].isin(['EASTERN EUROPE', 'WESTERN EUROPE', 'BALTICS'])]
factbook_europe.head()

,Country,Region,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service
1,Albania,EASTERN EUROPE,3581655,28748,124.6,1.26,-4.93,21.52,4500.0,86.5,71.2,21.09,4.42,74.49,3.0,15.11,5.22,0.232,0.188,0.579
4,Andorra,WESTERN EUROPE,71201,468,152.1,0.00,6.60,4.05,19000.0,100.0,497.2,2.22,0.00,97.78,3.0,8.71,6.25,NaN,NaN,NaN
12,Austria,WESTERN EUROPE,8192880,83870,97.7,0.00,2.00,4.66,30000.0,98.0,452.2,16.91,0.86,82.23,3.0,8.74,9.76,0.018,0.304,0.678
19,Belgium,WESTERN EUROPE,10379067,30528,340.0,0.22,1.23,4.68,29100.0,98.0,462.6,23.28,0.40,76.32,3.0,10.38,10.27,0.010,0.240,0.749
25,Bosnia & Herzegovina,EASTERN EUROPE,4498976,51129,88.0,0.04,0.31,21.05,6100.0,NaN,215.4,13.60,2.96,83.44,4.0,8.77,8.27,0.142,0.308,0.550


In [19]:
factbook_pivot_europe = factbook_europe.pivot_table(columns=['Country'])
factbook_pivot_europe.head()

Country,Albania,Andorra,Austria,Belgium,Bosnia & Herzegovina,Bulgaria,Croatia,Czech Republic,Denmark,Estonia,...,Portugal,Romania,San Marino,Serbia,Slovakia,Slovenia,Spain,Sweden,Switzerland,United Kingdom
Agriculture,0.232,NaN,0.018,0.01,0.142,0.093,0.07,0.034,0.018,0.04,...,0.053,0.101,NaN,0.166,0.035,0.028,0.04,0.011,0.015,0.005
Arable (%),21.090,2.22,16.910,23.28,13.600,40.020,26.09,39.800,54.020,16.04,...,21.750,40.820,16.67,33.350,30.160,8.600,26.07,6.540,10.420,23.460
Area (sq. mi.),28748.000,468.00,83870.000,30528.00,51129.000,110910.000,56542.00,78866.000,43094.000,45226.00,...,92391.000,237500.000,61.00,88361.000,48845.000,20273.000,504782.00,449964.000,41290.000,244820.000
Birthrate,15.110,8.71,8.740,10.38,8.770,9.650,9.61,9.020,11.130,10.04,...,10.720,10.700,10.02,NaN,10.650,8.980,10.06,10.270,9.710,10.710
Climate,3.000,3.00,3.000,3.00,4.000,3.000,NaN,3.000,3.000,3.00,...,3.000,3.000,NaN,NaN,3.000,NaN,3.00,3.000,3.000,3.000


In [20]:
data_europe = pd.concat([factbook_pivot_europe, happiness_pivot]).astype(object).T.dropna(subset=['Happiness.Rank']).dropna() #concatenating both pivot tables together
data_europe

,Agriculture,Arable (%),Area (sq. mi.),Birthrate,Climate,Coastline (coast/area ratio),Crops (%),Deathrate,GDP ($ per capita),Industry,...,Pop. Density (per sq. mi.),Population,Service,Dystopia.Residual,Family,Freedom,Generosity,Happiness.Rank,Health..Life.Expectancy.,Trust..Government.Corruption.
Albania,0.232,21.09,28748,15.11,3,1.26,4.42,5.22,4500,0.188,...,124.6,3.58166e+06,0.579,1.49044,0.803685,0.381499,0.201313,109,0.73116,0.0398642
Austria,0.018,16.91,83870,8.74,3,0,0.86,9.76,30000,0.304,...,97.7,8.19288e+06,0.678,2.13851,1.45994,0.567766,0.316472,13,0.815328,0.22106
Belgium,0.01,23.28,30528,10.38,3,0.22,0.4,10.27,29100,0.24,...,340,1.03791e+07,0.749,2.12421,1.46231,0.539771,0.231503,17,0.818092,0.251343
Bulgaria,0.093,40.02,110910,9.65,3,0.32,1.92,14.27,7600,0.304,...,66.6,7.38537e+06,0.603,0.996139,1.43438,0.289232,0.113178,105,0.708218,0.0110515
Czech Republic,0.034,39.8,78866,9.02,3,0,3.05,10.59,15700,0.393,...,129.8,1.02355e+07,0.573,2.45186,1.43389,0.490946,0.0881068,23,0.754444,0.0368729
Denmark,0.018,54.02,43094,11.13,3,16.97,0.19,10.36,31100,0.246,...,126.5,5.45066e+06,0.735,2.31371,1.55112,0.626007,0.35528,2,0.792566,0.40077
Estonia,0.04,16.04,45226,10.04,3,8.39,0.45,13.25,12300,0.294,...,29.3,1.32433e+06,0.666,1.35751,1.47667,0.479131,0.0988908,66,0.695168,0.183249
Finland,0.028,7.19,338145,10.45,3,0.37,0.03,9.86,27400,0.295,...,15.5,5.23137e+06,0.676,2.43018,1.54025,0.617951,0.245483,5,0.809158,0.382612
France,0.022,33.53,547030,11.99,4,0.63,2.07,9.14,27600,0.214,...,111.3,6.08761e+07,0.764,2.00595,1.38778,0.470222,0.129762,31,0.844466,0.172502
Germany,0.009,33.85,357021,8.25,3,0.67,0.59,10.62,27600,0.296,...,230.9,8.24223e+07,0.695,2.01577,1.47252,0.562511,0.336269,16,0.798951,0.276732


In [21]:
Happiness_Correlations_Europe = data_europe.astype(float).corr(method='pearson', min_periods=10)
Happiness_Correlations_Europe = Happiness_Correlations_Europe[['Happiness.Rank']]
Happiness_Correlations_Europe = Happiness_Correlations_Europe.abs().sort_values(by='Happiness.Rank', ascending=False)
Happiness_Correlations_Europe

,Happiness.Rank
Happiness.Rank,1.000000
GDP ($ per capita),0.839920
Dystopia.Residual,0.816807
Trust..Government.Corruption.,0.796182
Freedom,0.791811
Generosity,0.727384
Family,0.700464
Infant mortality (per 1000 births),0.659755
Agriculture,0.650629
Net migration,0.639366


Interesting to see that in Europe, the Life Expectancy is much lower related to happiness than worldwide. Possible reason might be that the life expectancy is fortunately so high in Europe that it doesn't matter anymore?

## Europe Graphs

In [22]:
plotly.offline.iplot({
    "data": [go.Scatter(x=data_europe['Happiness.Rank'], 
                        y=data_europe['GDP ($ per capita)'], 
                        mode="markers", 
                        hovertext=data_europe.index,
                        )
            ],
    "layout": go.Layout(title="Europe",
                        xaxis=dict(title='Happiness Rank'),
                        yaxis=dict(title='GDP per capita ($)')
                       )
})

In [23]:
plotly.offline.iplot({
    "data": [go.Scatter(x=data_europe['Happiness.Rank'], 
                        y=data_europe['Dystopia.Residual'], 
                        mode="markers", 
                        hovertext=data_europe.index,
                        )
            ],
    "layout": go.Layout(title="Europe",
                        xaxis=dict(title='Happiness Rank'),
                        yaxis=dict(title='Dystopia Residual')
                       )
})

In [24]:
plotly.offline.iplot({
    "data": [go.Scatter(x=data_europe['Happiness.Rank'], 
                        y=data_europe['Trust..Government.Corruption.'], 
                        mode="markers", 
                        hovertext=data_europe.index,
                        )
            ],
    "layout": go.Layout(title="Europe",
                        xaxis=dict(title='Happiness Rank'),
                        yaxis=dict(title='Corruption')
                       )
})

## Latin America

In [25]:
factbook.Region.unique()

array(['ASIA (EX. NEAR EAST)', 'EASTERN EUROPE', 'NORTHERN AFRICA',
       'OCEANIA', 'WESTERN EUROPE', 'SUB-SAHARAN AFRICA',
       'LATIN AMER. & CARIB', 'C.W. OF IND. STATES', 'NEAR EAST',
       'NORTHERN AMERICA', 'BALTICS'], dtype=object)

In [26]:
factbook_latinamerica = factbook.loc[factbook['Region'].isin(['LATIN AMER. & CARIB'])]
factbook_latinamerica.head()

,Country,Region,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service
6,Anguilla,LATIN AMER. & CARIB,13477,102,132.1,59.80,10.76,21.03,8600.0,95.0,460.0,0.00,0.00,100.00,2.0,14.17,5.34,0.040,0.180,0.780
7,Antigua & Barbuda,LATIN AMER. & CARIB,69108,443,156.0,34.54,-6.15,19.46,11000.0,89.0,549.9,18.18,4.55,77.27,2.0,16.93,5.37,0.038,0.220,0.743
8,Argentina,LATIN AMER. & CARIB,39921833,2766890,14.4,0.18,0.61,15.18,11200.0,97.1,220.4,12.31,0.48,87.21,3.0,16.73,7.55,0.095,0.358,0.547
10,Aruba,LATIN AMER. & CARIB,71891,193,372.5,35.49,0.00,5.89,28000.0,97.0,516.1,10.53,0.00,89.47,2.0,11.03,6.68,0.004,0.333,0.663
14,"Bahamas, The",LATIN AMER. & CARIB,303770,13940,21.8,25.41,-2.20,25.21,16700.0,95.6,460.6,0.80,0.40,98.80,2.0,17.57,9.05,0.030,0.070,0.900


In [27]:
factbook_pivot_latinamerica = factbook_latinamerica.pivot_table(columns=['Country'])
factbook_pivot_latinamerica

Country,Anguilla,Antigua & Barbuda,Argentina,Aruba,"Bahamas, The",Barbados,Belize,Bolivia,Brazil,British Virgin Is.,...,Puerto Rico,Saint Kitts & Nevis,Saint Lucia,Saint Vincent and the Grenadines,Suriname,Trinidad & Tobago,Turks & Caicos Is,Uruguay,Venezuela,Virgin Islands
Agriculture,0.04,0.038,9.500000e-02,0.004,0.03,0.06,0.142,0.128,8.400000e-02,0.018,...,0.01,0.035,0.07,0.10,0.13,0.007,NaN,0.093,4.000000e-02,0.01
Arable (%),0.00,18.180,1.231000e+01,10.530,0.80,37.21,2.850,2.670,6.960000e+00,20.000,...,3.95,19.440,6.56,17.95,0.37,14.620,2.33,7.430,2.950000e+00,11.76
Area (sq. mi.),102.00,443.000,2.766890e+06,193.000,13940.00,431.00,22966.000,1098580.000,8.511965e+06,153.000,...,13790.00,261.000,616.00,389.00,163270.00,5128.000,430.00,176220.000,9.120500e+05,1910.00
Birthrate,14.17,16.930,1.673000e+01,11.030,17.57,12.71,28.840,23.300,1.656000e+01,14.890,...,12.77,18.020,19.68,16.18,18.02,12.900,21.84,13.910,1.871000e+01,13.96
Climate,2.00,2.000,3.000000e+00,2.000,2.00,2.00,2.000,1.500,2.000000e+00,2.000,...,2.00,2.000,2.00,2.00,2.00,2.000,2.00,3.000,2.000000e+00,2.00
Coastline (coast/area ratio),59.80,34.540,1.800000e-01,35.490,25.41,22.51,1.680,0.000,9.000000e-02,52.290,...,3.63,51.720,25.65,21.59,0.24,7.060,90.47,0.370,3.100000e-01,9.84
Crops (%),0.00,4.550,4.800000e-01,0.000,0.40,2.33,1.710,0.190,9.000000e-01,6.670,...,5.52,2.780,22.95,17.95,0.06,9.160,0.00,0.230,9.200000e-01,2.94
Deathrate,5.34,5.370,7.550000e+00,6.680,9.05,8.67,5.720,7.530,6.170000e+00,4.420,...,7.65,8.330,5.08,5.98,7.27,10.570,4.21,9.050,4.920000e+00,6.43
GDP ($ per capita),8600.00,11000.000,1.120000e+04,28000.000,16700.00,15700.00,4900.000,2400.000,7.600000e+03,16000.000,...,16800.00,8800.000,5400.00,2900.00,4000.00,9500.000,9600.00,12800.000,4.800000e+03,17200.00
Industry,0.18,0.220,3.580000e-01,0.333,0.07,0.16,0.152,0.352,4.000000e-01,0.062,...,0.45,0.258,0.20,0.26,0.22,0.570,NaN,0.311,4.190000e-01,0.19


In [28]:
data_latinamerica = pd.concat([factbook_pivot_latinamerica, happiness_pivot]).astype(object).T.dropna(subset=['Happiness.Rank']).dropna() #concatenating both pivot tables together
data_latinamerica.index

Index(['Argentina', 'Belize', 'Bolivia', 'Brazil', 'Chile', 'Colombia',
       'Costa Rica', 'Dominican Republic', 'Ecuador', 'El Salvador',
       'Guatemala', 'Haiti', 'Honduras', 'Jamaica', 'Mexico', 'Nicaragua',
       'Panama', 'Paraguay', 'Peru', 'Uruguay', 'Venezuela'],
      dtype='object')

In [29]:
Happiness_Correlations_LatinAmerica = data_latinamerica.astype(float).corr(method='pearson', min_periods=10)
Happiness_Correlations_LatinAmerica = Happiness_Correlations_LatinAmerica[['Happiness.Rank']]
Happiness_Correlations_LatinAmerica = Happiness_Correlations_LatinAmerica.abs().sort_values(by='Happiness.Rank', ascending=False)
Happiness_Correlations_LatinAmerica

,Happiness.Rank
Happiness.Rank,1.000000
Dystopia.Residual,0.777581
Health..Life.Expectancy.,0.730379
Phones (per 1000),0.701684
Infant mortality (per 1000 births),0.678366
Freedom,0.662075
GDP ($ per capita),0.649866
Birthrate,0.645751
Literacy (%),0.583725
Family,0.582315


## Latin America Graphs

In [30]:
plotly.offline.iplot({
    "data": [go.Scatter(x=data_latinamerica['Happiness.Rank'], 
                        y=data_latinamerica['Dystopia.Residual'], 
                        mode="markers", 
                        hovertext=data_latinamerica.index,
                        )
            ],
    "layout": go.Layout(title="Latin America",
                        xaxis=dict(title='Happiness Rank'),
                        yaxis=dict(title='Dystopia Residual')
                       )
})

In [31]:
plotly.offline.iplot({
    "data": [go.Scatter(x=data_latinamerica['Happiness.Rank'], 
                        y=data_latinamerica['Health..Life.Expectancy.'], 
                        mode="markers", 
                        hovertext=data_latinamerica.index,
                        )
            ],
    "layout": go.Layout(title="Latin America",
                        xaxis=dict(title='Happiness Rank'),
                        yaxis=dict(title='Life Expectancy')
                       )
})

In [32]:
plotly.offline.iplot({
    "data": [go.Scatter(x=data_latinamerica['Happiness.Rank'], 
                        y=data_latinamerica['Phones (per 1000)'], 
                        mode="markers", 
                        hovertext=data_latinamerica.index,
                        )
            ],
    "layout": go.Layout(title="Latin America",
                        xaxis=dict(title='Happiness Rank'),
                        yaxis=dict(title='Phones per 1000')
                       )
})

# World Maps

In [33]:
graph_world = [ dict(
        type = 'choropleth',
        locations = data_plot.index,
        locationmode = 'country names',
        z = data_plot['Happiness.Rank'],
        text = data_plot.index,
        colorscale = 'RdBu',
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.3
            ) ),
        hoverinfo = 'z+text'      
      ) ]

layout = dict(
    title = 'Happiness Rank',
    autosize=False,
    width=1000,
    height=900,
    geo = dict(
        showframe = False,
        showcoastlines = True,
        projection = dict(
            type = 'mercator'
        )
    )
)

plotly.offline.iplot(dict(data=graph_world, layout=layout))

In [34]:
graph_world_gdp = [ dict(
        type = 'choropleth',
        locations = data_plot.index,
        locationmode = 'country names',
        z = data_plot['GDP ($ per capita)'],
        text = data_plot.index,
        colorscale = 'Blues',
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        hoverinfo = 'z+text'      
      ) ]

layout = dict(
    title = 'GDP per Capita (in $)',
    autosize=False,
    width=1000,
    height=900,
    geo = dict(
        showframe = False,
        showcoastlines = True,
        projection = dict(
            type = 'mercator'
        )
    )
)

plotly.offline.iplot(dict(data=graph_world_gdp, layout=layout))

In [35]:
graph_world_fam = [ dict(
        type = 'choropleth',
        locations = data_plot.index,
        locationmode = 'country names',
        z = data_plot['Family'],
        text = data_plot.index,
        colorscale = 'RdBu',
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        hoverinfo = 'z+text'      
      ) ]

layout = dict(
    title = 'Family',
    autosize=False,
    width=1000,
    height=900,
    geo = dict(
        showframe = False,
        showcoastlines = True,
        projection = dict(
            type = 'mercator'
        )
    )
)

plotly.offline.iplot(dict(data=graph_world_fam, layout=layout))

In [36]:
graph_world_fre = [ dict(
        type = 'choropleth',
        locations = data_plot.index,
        locationmode = 'country names',
        z = data_plot['Freedom'],
        text = data_plot.index,
        colorscale = 'RdBu',
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        hoverinfo = 'z+text'      
      ) ]

layout = dict(
    title = 'Freedom',
    autosize=False,
    width=1000,
    height=900,
    geo = dict(
        showframe = False,
        showcoastlines = True,
        projection = dict(
            type = 'mercator'
        )
    )
)

plotly.offline.iplot(dict(data=graph_world_fre, layout=layout))

In [37]:
graph_world_bir = [ dict(
        type = 'choropleth',
        locations = data_plot.index,
        locationmode = 'country names',
        z = data_plot['Birthrate'],
        text = data_plot.index,
        colorscale = 'Blues',
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        hoverinfo = 'z+text'      
      ) ]

layout = dict(
    title = 'Birthrate',
    autosize=False,
    width=1000,
    height=900,
    geo = dict(
        showframe = False,
        showcoastlines = True,
        projection = dict(
            type = 'mercator'
        )
    )
)

plotly.offline.iplot(dict(data=graph_world_bir, layout=layout))